In [5]:
from pyld import jsonld 
import cmipld
import os, re, glob

In [6]:
import requests
# requests.get('https://wcrp-cmip.github.io/WCRP-UNIVERSE/graph').json()

In [7]:
# jsonld.frame('https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/graph.jsonld',{"@id":"atmos"})

In [8]:
# jsonld.frame('https://wcrp-cmip.github.io/WCRP-UNIVERSE/graph.jsonld',{"@context":"https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos","@id":"https://wcrp-cmip.github.io/WCRP-UNIVERSE/realm/atmos"})

------

In [9]:
import cmipld

rdfvalues = []
for p,v in cmipld.mapping.items():
    print(p)
    try:
        rdf = jsonld.to_rdf(v+'graph.jsonld')
        rdfvalues.extend(list(rdf.values()))
    except Exception as e:
        print('failed',v)
        print(e)

wcrp-universe
mip-variables
cmip6plus
cmip7
cf
obs4mips
failed https://wcrp-cmip.github.io/obs4MIPs-cmor-tables-ld/
('Could not expand input before serialization to RDF.',)
Type: jsonld.RdfError
Cause: ('Could not retrieve a JSON-LD document from the URL.',)
Type: jsonld.LoadDocumentError
Code: loading document failed
Cause: Expecting value: line 1 column 1 (char 0)  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/pyld/documentloader/requests.py", line 72, in loader
    'document': response.json()
  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/requests/models.py", line 978, in json
    raise RequestsJSONDecodeError(e.msg, e.doc, e.pos)
  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/base/lib/python3.10/site-packages/pyld/jsonld.py", line 1219, in to_rdf
    expanded = self.expand(input_, options)
  File "/Users/daniel.ellis/customlib/homebrew/Caskroom/mambaforge/b

In [18]:
# jsonld.to_rdf('https://wcrp-cmip.github.io/MIP-variables/graph.jsonld')

In [19]:
rdfnodes = []
rdflinks = []

for g in rdfvalues:
    for t in g:
        if not t:
            continue
        s = str(t)
        if 'literal' not in s:
            if 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type' in s:
                # self._add_type_node(t, type_map, clink, prefix)
                # print('node',t)
                rdfnodes.append(t['subject']['value'])
            else:
                # self._add_link(t, clink) 
                rdflinks.append({'source':t['subject']['value'],'target':t['object']['value'],'type':t['predicate']['value']})

In [20]:
len(rdfvalues)

23

In [28]:
from collections import Counter
Counter(rdfnodes)
import pandas as pd
df = pd.DataFrame(rdflinks)
# remove the website part
# df = df.map(lambda x: x.split('github.io/')[1])
def prefixify(x):
    for m in cmipld.mapping.items():
        if x.startswith(m[1]):
            x = x.replace(m[1],m[0]+':') 
    return x


# df[df.source[0] != '_']
keep = [i[1].source[0] != '_' and i[1].target[0] != '_' for i in df.iterrows()]

df = df[keep]

df = df.map(prefixify)


    

In [29]:
df.groupby('type').count()

,source,target
type,,
cmip6plus:experiment/activity,28,28
cmip6plus:experiment/model_realms,200,200
cmip6plus:experiment/parameters,50,50
cmip6plus:experiment/parent_experiment,50,50
cmip6plus:project/drs,1,1
cmip6plus:project/index,1,1
cmip6plus:project/mip_era,1,1
cmip6plus:project/product,1,1
cmip6plus:project/required,1,1


In [30]:

short = lambda x: x.split('/')[0] if 'https' not in x else x

df['source_base'] = df['source'].map(short)
df['target_base'] = df['target'].map(short)

In [31]:
df.groupby('source_base').count()

,source,target,type,target_base
source_base,,,,
cmip6plus:experiment,328,328,328,328
cmip6plus:project,83,83,83,83
cmip6plus:source,12,12,12,12
cmip7:experiment,42,42,42,42
mip-variables:tables,2126,2126,2126,2126
wcrp-universe:activity,27,27,27,27
wcrp-universe:institutions,72,72,72,72
wcrp-universe:linktest,2,2,2,2


In [32]:
nodes = []
for n in set(map(short,map(prefixify,rdfnodes))):
    in_weight = df[df['source_base']==n].shape[0]
    out_weight = df[df['target_base']==n].shape[0]
    url = n.split(':')
    try:
        url[0] = cmipld.mapping[url[0]]
    except:
        print('failed',url)
    
    if n[0]=='_':continue
    
    nodes.append({'id':n,'in_weight':in_weight,'out_weight':out_weight,'total_weight':in_weight+out_weight,'category':n.split(':')[0],'href':''.join(url)})

nodesdf = pd.DataFrame(nodes)
isolates = nodesdf[nodesdf['total_weight']==0]['id'].values
print('Warning isolates:', isolates)



nodes

Warning isolates: ['cmip6plus:sub_experiment' 'wcrp-universe:consortia'
 'wcrp-universe:source_type' 'wcrp-universe:resolution'
 'wcrp-universe:frequency']


[{'id': 'cmip6plus:experiment',
  'in_weight': 328,
  'out_weight': 100,
  'total_weight': 428,
  'category': 'cmip6plus',
  'href': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment'},
 {'id': 'cmip6plus:sub_experiment',
  'in_weight': 0,
  'out_weight': 0,
  'total_weight': 0,
  'category': 'cmip6plus',
  'href': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/sub_experiment'},
 {'id': 'wcrp-universe:consortia',
  'in_weight': 0,
  'out_weight': 0,
  'total_weight': 0,
  'category': 'wcrp-universe',
  'href': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/consortia'},
 {'id': 'mip-variables:tables',
  'in_weight': 2126,
  'out_weight': 78,
  'total_weight': 2204,
  'category': 'mip-variables',
  'href': 'https://wcrp-cmip.github.io/MIP-variables/tables'},
 {'id': 'wcrp-universe:source_type',
  'in_weight': 0,
  'out_weight': 0,
  'total_weight': 0,
  'category': 'wcrp-universe',
  'href': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/source_type'},
 {'id': 'wcrp-universe:license',
  'in_weig

In [33]:
links = []
for l,w in Counter(['->'.join(i) for i in df[['source_base','type','target_base']].values]).items():
    split = l.split('->')
    links.append({'source':split[0],'target':split[-1],'type':split[1].replace(split[0],'')[1:], 'value':w})
    
links

[{'source': 'wcrp-universe:institutions',
  'target': 'wcrp-universe:location',
  'type': 'location',
  'value': 72},
 {'source': 'wcrp-universe:linktest',
  'target': 'wcrp-universe:linktest',
  'type': 'realm2',
  'value': 2},
 {'source': 'mip-variables:tables',
  'target': 'wcrp-universe:product',
  'type': 'product',
  'value': 77},
 {'source': 'mip-variables:tables',
  'target': 'mip-variables:variables',
  'type': 'variable_entry',
  'value': 2049},
 {'source': 'cmip6plus:source',
  'target': 'wcrp-universe:license',
  'type': 'license',
  'value': 12},
 {'source': 'wcrp-universe:activity',
  'target': 'cmip6plus:source',
  'type': 'source_activity',
  'value': 27},
 {'source': 'cmip6plus:experiment',
  'target': 'wcrp-universe:activity',
  'type': 'activity',
  'value': 28},
 {'source': 'cmip6plus:experiment',
  'target': 'wcrp-universe:realm',
  'type': 'model_realms',
  'value': 200},
 {'source': 'cmip6plus:experiment',
  'target': 'cmip6plus:experiment',
  'type': 'parameters

In [41]:
prefixes = list(cmipld.mapping.keys())
prefixes

['wcrp-universe', 'mip-variables', 'cmip6plus', 'cmip7', 'cf']

In [42]:
file = dict(nodes=nodes,links=links,prefixes=prefixes)

file

{'nodes': [{'id': 'cmip6plus:experiment',
   'in_weight': 328,
   'out_weight': 100,
   'total_weight': 428,
   'category': 'cmip6plus',
   'href': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/experiment'},
  {'id': 'cmip6plus:sub_experiment',
   'in_weight': 0,
   'out_weight': 0,
   'total_weight': 0,
   'category': 'cmip6plus',
   'href': 'https://wcrp-cmip.github.io/CMIP6Plus_CVs/sub_experiment'},
  {'id': 'wcrp-universe:consortia',
   'in_weight': 0,
   'out_weight': 0,
   'total_weight': 0,
   'category': 'wcrp-universe',
   'href': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/consortia'},
  {'id': 'mip-variables:tables',
   'in_weight': 2126,
   'out_weight': 78,
   'total_weight': 2204,
   'category': 'mip-variables',
   'href': 'https://wcrp-cmip.github.io/MIP-variables/tables'},
  {'id': 'wcrp-universe:source_type',
   'in_weight': 0,
   'out_weight': 0,
   'total_weight': 0,
   'category': 'wcrp-universe',
   'href': 'https://wcrp-cmip.github.io/WCRP-UNIVERSE/source_type'},
  {'

In [45]:
import json
json.dump(file,open('network.json','w'))